## Task 1 - Third-Order Letter Approximation Model

The below code takes in all of the txt files in texts/, filtering out special & numerical characters, then counting how often each sequence of three characters appears and compiles it into a dictionary.

<hr>

In [48]:
import os
import re
import random

texts = []
sanitizedTexts = []

for title in os.scandir("texts"): # [2]
    if((title.name.find(".txt")) != -1): # Only attempt to open + add the text file if it's a .txt
        fileContent = open(title) # [1]
        texts.append(fileContent.read()) # Add it to the texts list.
        fileContent.close()

for t in texts:
    #sanitizedTexts.append(sanitizeText(t))
    print(sanitizeText(t))

#produceTrigrams(sanitizedTexts)
        

FRANKENSTEINOR THE MODERN PROMETHEUSBY MARY WOLLSTONECRAFT GODWIN SHELLEY CONTENTS LETTER  LETTER  LETTER  LETTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER  CHAPTER LETTER TO MRS. SAVILLE ENGLAND.ST. PETERSBURGH DEC. TH .YOU WILL REJOICE TO HEAR THAT NO DISASTER HAS ACCOMPANIED THECOMMENCEMENT OF AN ENTERPRISE WHICH YOU HAVE REGARDED WITH SUCH EVILFOREBODINGS. I ARRIVED HERE YESTERDAY AND MY FIRST TASK IS TO ASSUREMY DEAR SISTER OF MY WELFARE AND INCREASING CONFIDENCE IN THE SUCCESSOF MY UNDERTAKING.I AM ALREADY FAR NORTH OF LONDON AND AS I WALK IN THE STREETS OFPETERSBURGH I FEEL A COLD NORTHERN BREEZE PLAY UPON MY CHEEKS WHICHBRACES MY NERVES AND FILLS ME WITH DELIGHT. DO YOU UNDERSTAND THISFEELING THIS BREEZE WHICH HAS TRAVELLED FROM THE REGIONS TOWARDSWHICH I AM ADVANCING GIVES ME A FORETASTE OF THOSE ICY CLIMES.INSPIRI

In [47]:
def sanitizeText(text):
    preamble = " ***" # Ending of the preamble
    postamble = "*** END OF " # Beginning of the postamble.

    # [3] Strips out the preamble and postamble from a text by
    # creating a string in between the preamble and postamble variables.
    sanitizedText = (t[t.index(preamble)+len(preamble):t.index(postamble)]) 
    sanitizedText = re.sub("[^a-zA-Z\\s.\n]", "", sanitizedText) # [4]

    return sanitizedText.upper().rstrip().strip().replace("\n", "") # Set to uppercase, remove new lines, and remove trailing spaces.

In [35]:
def produceTrigrams(texts):
    trigrams = {}

    #for text in texts:
       # for counter, c in enumerate(text):
            #if(c != "\n"):
                #print(text[counter:counter+3])
                

## Task 2 - Third-Order Letter Approximation Generation

<hr>

## Task 3 - Analyze The Model

<hr>

## Task 4 - Export Model

<hr>

## References

- [1] - Reading text files in Python: https://www.w3schools.com/python/python_file_open.
- [2] - Using Python's OS Module: https://www.geeksforgeeks.org/how-to-iterate-over-files-in-directory-using-python/
- [3] - Finding a string between two substrings (used for removing Pre/Postamble): https://stackoverflow.com/a/51456576
- [4] - Using regex in Python: https://www.w3schools.com/python/python_regex.asp
- [5] - Slicing text in Python: https://stackoverflow.com/a/509295
- 
- 

<hr>